## AntiAircraftArtillery

Given a fixed position on the ground, calculates azimuth, elevation and distance to nearby (<35km) aircraft.

***Do not use to effectively shot down planes, please!***

Libraries needed:

In [1]:
import numpy as np
from requests import get
from json import loads
from pandas import Series
from pandas.io.json import json_normalize

#### Function that calculates [ECEF](https://en.wikipedia.org/wiki/ECEF) coordinates:

<img src=../images/coor_geode_geocen.jpg width="25%">

The equations to calculate ECEF coordinates of a point of latitude $\varphi$, longitude $\lambda$ and altitude $h$:

\begin{align*}
    \left[
        \begin{array}{c}
            x\\
            y\\
            z\\
        \end{array}
    \right]
  =
    \left[
        \begin{array}{c}
            (N + h)cos\varphi cos\lambda\\
            (N + h)cos\varphi sin\lambda\\
            \big(N(\frac{b^2}{a^2}) + h\big)sin\varphi\\
        \end{array}
    \right]
\end{align*}

where:

\begin{align*} 
    \begin{array}{l}
            N = \frac{a}{\sqrt{1-e^2sin^2(\varphi)}}\\
            a = Earth \hspace{1mm} equatorial \hspace{1mm} axis\\
            b = Earth \hspace{1mm} polar \hspace{1mm} axis\\
            e = \sqrt{1-\frac{b^2}{a^2}}\\
    \end{array}
\end{align*}


[`N`](https://en.wikipedia.org/wiki/Earth_radius#Prime_vertical) is radius of curvature in the prime vertical and `e` the eccentricity. According to the [WGS84](https://en.wikipedia.org/wiki/World_Geodetic_System#WGS84) model:
* a = 6378137 m
* b = 6356752.3142 m
* e = 0.081819190842622

more [info](http://earth-info.nga.mil/GandG/publications/tr8350.2/wgs84fin.pdf) on the formulae and data. Therefore, the distance will be:

$$ d = ||\vec{ECEF}_{aircraft}- \vec{ECEF}_{ref}||$$

In [2]:
def geocentric_coordinates(lat, long, h):
    """
    Calculates ECEF coordinates (Earth Centered, Earth Fixed) in meters.
    
    Input:
        lat, long - latitude y longitude in degrees
        h - altitude above SL in feet
        
    Output:
         - array with ECEF coordinates in meters
    """
    a = 6378137  # [m] Earth equatorial axis 
    b = 6356752.3142  # [m] Earth polar axis
    e = 0.081819190842622  # Earth eccentricity
    
    lat = np.radians(lat)  # degrees to radians
    long = np.radians(long) # degrees to radians
    h = h * 0.3048  # feets to meters
    
    N = a / (1 - (e * np.sin(lat))**2)**(.5)

    x = (N + h) * np.cos(lat) * np.cos(long)
    y = (N + h) * np.cos(lat) * np.sin(long)
    z = (((b/a)**2) * N + h) * np.sin(lat)
    
    return np.array([x, y, z])

#### Function that calculates distance, azimuth and elevation of an aircraft with respect a reference point:

In [3]:
def dist_az_elev(lat, long, h, lat_ref, long_ref, h_ref):
    """
    Returns unit vector from aircraft to the reference point, and its module in km.
    
    Input:
        lat, long - aircraft latitude and longitude in degrees
        h - aircraft altitude above sea level in feet
        lat_ref, long_ref - reference point latitude and longitude in degrees
        h_ref - reference point altitude above sea level in feet
        
    Output:
         - distance aircraft -> reference point in km, azimuth and elevation in degrees
    """
    v = geocentric_coordinates(lat, long, h) - \
        geocentric_coordinates(lat_ref, long_ref, h_ref)
    
    unit_vector_ecef = v / np.linalg.norm(v)
    distancia = np.linalg.norm(v) / 1e3  # [km]

    sla, cla = np.sin(np.radians(lat_ref)), np.cos(np.radians(lat_ref))
    slo, clo = np.sin(np.radians(long_ref)), np.cos(np.radians(long_ref))
    
    Lht = np.array([[-sla * clo, -sla * slo, cla],
                    [-slo,       clo,        0],
                    [-cla * clo, -cla * slo, -sla]])
    
    unit_vector_ned = np.dot(Lht, unit_vector_ecef)
    azimut = np.arctan2(unit_vector_ned[1], unit_vector_ned[0])
    altura = np.arctan(-unit_vector_ned[2] / np.sqrt(unit_vector_ned[0]**2 + unit_vector_ned[1]**2))
    
    return Series([distancia, np.degrees(azimut), np.degrees(altura)])

### Data extraction from [adsbexchange](https://adsbexchange.com/)

From the [Wikipedia](https://en.wikipedia.org/wiki/Automatic_dependent_surveillance_%E2%80%93_broadcast), "(...) **ADS–B** is a surveillance technology in which an aircraft determines its position via satellite navigation and periodically broadcasts it, enabling it to be tracked."

<img src=../images/adsb.jpg width="40%">

In [4]:
# header for the request
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko)'}  

# we are going to refine a little bit the window, to avoid getting all aircraft around the world
url = 'http://public-api.adsbexchange.com/VirtualRadar/AircraftList.json?lat=40.3&lng=-3.78&fDstL=0&fDstU=50'

result = get(url, headers=headers)  # give me my json!!

data_raw = result.content.decode()  # json decodification
data_json = loads(data_raw)  # json loaded!

<img src=../images/spain-lat-long.jpg width="40%">

And now it's Pandas time!

In [5]:
df_raw = json_normalize(data_json['acList'])  # aircraft list normalized
df = df_raw[np.isfinite(df_raw['Lat']) | np.isfinite(df_raw['Long'])]  # get only those with lat and long data

df = df[['GAlt', 'Spd', 'Lat', 'Long', 'Vsi', 'Trak', 
         'Reg', 'Call', 'Type', 'From', 'To']].set_index('Call')  # select the info we are interested in
df

,GAlt,Spd,Lat,Long,Vsi,Trak,Reg,Type,From,To
Call,,,,,,,,,,
IBE34HK,3362,166.7,40.547697,-3.557261,2304,12.1,EC-LXQ,A320,NaN,NaN
IBS3948,5912,206.0,40.593750,-3.627075,1024,312.0,EC-LYM,A320,NaN,NaN
AEA23VE,7039,275.9,40.648576,-3.476112,2880,40.4,EC-MUZ,B738,NaN,NaN
IBE0440,8912,269.4,40.686004,-3.673616,2752,7.7,EC-HUH,A321,"LEMD Madrid Barajas, Spain","LEBB Bilbao, Spain"
RYR7X,18037,348.1,40.635361,-4.003906,2944,268.7,EI-FTM,B738,"LIME Bergamo / Orio Al Serio, Italy","EDDH Hamburg, Germany"
IBE31DG,23438,395.1,40.428058,-4.314443,1536,241.7,EC-MXY,A20N,"LEMD Madrid Barajas, Spain","EGLL London Heathrow, United Kingdom"
TUI7HW,35991,442.3,40.633896,-3.910400,64,24.0,D-ATUZ,B738,"EDDV Hannover, Germany","GCLP Gran Canaria, Gran Canaria Island, Spain"
TVF91QK,4466,229.1,40.294360,-3.385162,-896,331.3,F-GZHV,B738,NaN,NaN
BEL9AD,16216,335.3,40.525026,-4.087226,-1088,159.9,OO-TCQ,A320,"EBBR Brussels, Belgium","LEMD Madrid Barajas, Spain"


## Now...AIM!

We will place our gun in the center of Madrid:

In [6]:
# Madrid coordinates as point of reference
lat_Mad, long_Mad, h_Mad = 40.4169, -3.7032, 622/0.3048  # lat, long, altitude [feet]

# lambda magic: apply the dist_az_elev function to our list of aircrafts
df[['Distance', 'Azimut', 'Elevation']] = df.apply(lambda df: dist_az_elev(df['Lat'], df['Long'], df['GAlt'], 
                                                                           lat_Mad, long_Mad, h_Mad), axis=1)

# now and finally, select those within 35km radius
df[(df['Distance'] < 35)].sort_values(by=['Distance'])

,GAlt,Spd,Lat,Long,Vsi,Trak,Reg,Type,From,To,Distance,Azimut,Elevation
Call,,,,,,,,,,,,,
WZZ577,2187,158.2,40.451148,-3.513503,-960,322.2,HA-LPX,A320,"LHBP Budapest Ferenc Liszt, Hungary","LIRN Napoli / Capodichino, Italy",16.540821,76.644954,0.080284
NaN,2937,150.4,40.423636,-3.485600,-768,322.3,OE-IJB,A320,NaN,NaN,18.486630,87.610063,0.763855
IBE34HK,3362,166.7,40.547697,-3.557261,2304,12.1,EC-LXQ,A320,NaN,NaN,19.087395,40.382590,1.123235
IBS3948,5912,206.0,40.593750,-3.627075,1024,312.0,EC-LYM,A320,NaN,NaN,20.708627,18.164014,3.173438
IBE0440,8912,269.4,40.686004,-3.673616,2752,7.7,EC-HUH,A321,"LEMD Madrid Barajas, Spain","LEBB Bilbao, Spain",30.068470,4.783774,3.859059
TVF91QK,4466,229.1,40.294360,-3.385162,-896,331.3,F-GZHV,B738,NaN,NaN,30.263856,116.628879,1.263877
TUI7HW,35991,442.3,40.633896,-3.910400,64,24.0,D-ATUZ,B738,"EDDV Hannover, Germany","GCLP Gran Canaria, Gran Canaria Island, Spain",31.584554,-36.010739,18.990889
AEA23VE,7039,275.9,40.648576,-3.476112,2880,40.4,EC-MUZ,B738,NaN,NaN,32.168417,36.718443,2.570069
BEL9AD,16216,335.3,40.525026,-4.087226,-1088,159.9,OO-TCQ,A320,"EBBR Brussels, Belgium","LEMD Madrid Barajas, Spain",34.992667,-69.637639,6.936780
